<a href="https://colab.research.google.com/github/boyu571/TIL/blob/master/DL_%EC%9D%B4%EB%A1%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#python 버전 확인
!python --version

Python 3.7.13


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.8.2


In [ ]:
# !pip install tensorflow==2.8.2

In [ ]:
# tensorflow 2.0 장점
# keras와 즉시 실행을 이용해서 쉬운 모델 작성 가능

In [ ]:
# 12시간 제한
import torch

##Linear Layer

In [ ]:
w = torch.FloatTensor([[1,2],
                       [3,4],
                       [5,6]])
b = torch.FloatTensor([2,2])

In [ ]:
def linear(x, W, b):
  y = torch.matmul(x,W) + b

  return y

In [ ]:
x = torch.FloatTensor(4,3)

In [ ]:
y = linear(x, W, b)
print(y.size())

NameError: ignored

파이토치에는 nn(neural networks) 패키지가 있다. nn.Module을 상속받은 MyLinear라는 클래스를 정의. nn.Module을 상속받은 클래스는 보통 2개의 메서드 __init__과 forward 를 오버라이드합니다. __init__ 함수는 계층 내부에서 필요한 변수를 미리 선언하는 부분이며, 심지어 또 다른 계층(nn.Module을 상속받은 클래스의 객체)을 소유하도록 할 수도 있습니다. forward 함수는 계층을 통과하는데 필요한 계산을 수행하도록 구현하는 부분입니다.

In [ ]:
import torch.nn as nn

In [ ]:
class MyLinear(nn.Module):

  def __init__(self, input_dim = 3, output_dim = 2):
    self.input_dim = input_dim
    self.output_dim = output_dim
    
    super().__init__()

    self.W = torch.FloatTensor(input_dim, output_dim)
    self.b = torch.FloatTensor(output_dim)

  # You should override 'forward' method to implement detail.
  # The input arguments and outputs can be designed as you wish.
  
  def forward(self, x):
    # |x| = (batch_size, input_dim)
    y = torch.matmul(x, self.W) + self.b
    # |y| = (batch_size, input_dim) * (input_dim, output_dim)
    #     = (batch_size, output_dim)

    return y

In [ ]:
linear = MyLinear(3, 2)

y = linear(x)

nn.Module을 상속받은 객체는 __call__함수와 forward가 맵핑되어 있어 forward를 직접 부를 필요가 없다. forward 함수를 따로 호출하지 않고, 객체명에 바로 괄호를 열어 tensor x를 인수로 넘겨주었다는 것.
단, 내부에 학습할 수 있는 파라미터는 없는 것으로 인식.

In [ ]:
for p in linear.parameters():
  print(p)

W와 b를 파이토칯에서 학습 가능하도록 인식할 수 있는 파라미터로 만들어주어야 함. torch.nn.Parameter 클래스를 활용.

In [ ]:
class MyLinear(nn.Module):

  def __init__(self, input_dim = 3, output_dim = 2):
    self.input_dim = input_dim
    self.output_dim = output_dim
    
    super().__init__()

    self.W = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
    self.b = nn.Parameter(torch.FloatTensor(output_dim))

  # You should override 'forward' method to implement detail.
  # The input arguments and outputs can be designed as you wish.
  
  def forward(self, x):
    # |x| = (batch_size, input_dim)
    y = torch.matmul(x, self.W) + self.b
    # |y| = (batch_size, input_dim) * (input_dim, output_dim)
    #     = (batch_size, output_dim)

    return y

In [ ]:
linear = MyLinear(3, 2)

y = linear(x)

In [ ]:
for p in linear.parameters():
  print(p)

Parameter containing:
tensor([[9.5684e-33, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00]], requires_grad=True)
Parameter containing:
tensor([9.5688e-33, 0.0000e+00], requires_grad=True)


사실 torch.nn에 미리 정의된 선형 계층을 호출하면 간단

In [ ]:
linear = nn.Linear(3, 2)

y = linear(x)

In [ ]:
for p in linear.parameters():
  print(p)

Parameter containing:
tensor([[-0.3576, -0.0927, -0.0186],
        [-0.1692,  0.1588, -0.0810]], requires_grad=True)
Parameter containing:
tensor([ 0.1458, -0.2834], requires_grad=True)


In [ ]:
class MyLinear(nn.Module):

  def __init__(self, input_dim = 3, output_dim = 2):
    self.input_dim = input_dim
    self.output_dim = output_dim
    
    super().__init__()

    self.linear = nn.Linear(input_dim, output_dim)

  def forward(self, x):
    # |x| = (batch_size, input_dim)
    y = self.linear(x)
    # |y| = (batch_size, output_dim)

    return y

CUDA(Computed Unified Device Architecture)는 엔비디아 GPU에서 수행하는 병렬 연산을 프로그래밍 언어를 통해 구현할 수 있도록 하는 기술.
CUDA 함수를 통해 GPU의 메모리 상에 텐서 생성 가능

In [ ]:
x = torch.cuda.FloatTensor(2,2)
x

tensor([[0., 0.],
        [0., 0.]], device='cuda:0')

In [ ]:
x = torch.FloatTensor(2,2)
x

x = x.cuda()
x

tensor([[-7.4171e-07,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00]], device='cuda:0')

In [ ]:
x = x.cuda(device=1) # If you don't have 2nd gpu, error will be occurred.
x

RuntimeError: ignored

텐서는 device 속성을 가지고 있어, 해당 텐서가 위치한 디바이스를 쉽게 파악할 수 있다.

In [ ]:
x = torch.cuda.FloatTensor(2, 2)
x.device

device(type='cuda', index=0)

 nn.Module의 하위 클래스 객체는 해당 속성을 갖고 있지 않다. 따라서 만약 모델이 어느 장치에 올라가있는지 알고 싶다면 다음과 같은 방법을 취할 수 있다.

In [ ]:
layer = nn.Linear(2, 2)
next(layer.parameters()).device

device(type='cpu')